# Lab: Cross Validation and Bootstrap

In [2]:
import numpy as np 
import pandas as pd 

from matplotlib.pyplot import subplots 
from plotnine import *

import statsmodels.api as sm

from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)

from sklearn.model_selection import train_test_split

from functools import partial
from sklearn.model_selection import ( \
    cross_validate,
    KFold,
    ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [3]:
random_state=3

In [4]:
Auto = load_data('Auto') # We load the Auto dataset
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=random_state) # We split the data into traiing and validation sets. 
# There are 392 observations so we set test_size=196

In [5]:
Auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
name,,,,,,,,
chevrolet chevelle malibu,18.0,8,307.0,130,3504,12.0,70,1
buick skylark 320,15.0,8,350.0,165,3693,11.5,70,1
plymouth satellite,18.0,8,318.0,150,3436,11.0,70,1
amc rebel sst,16.0,8,304.0,150,3433,12.0,70,1
ford torino,17.0,8,302.0,140,3449,10.5,70,1


We fit a linear regression using only the observations corresponding to the training set `Auto_train`

In [6]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [7]:
X_train

,intercept,horsepower
name,,
datsun 810 maxima,1.0,120
buick electra 225 custom,1.0,225
datsun f-10 hatchback,1.0,70
volvo diesel,1.0,76
maxda glc deluxe,1.0,65
...,...,...
buick century special,1.0,105
chevrolet chevelle malibu classic,1.0,100
mercury monarch ghia,1.0,139


In [8]:
Auto_train['horsepower']

name
datsun 810 maxima                    120
buick electra 225 custom             225
datsun f-10 hatchback                 70
volvo diesel                          76
maxda glc deluxe                      65
                                    ... 
buick century special                105
chevrolet chevelle malibu classic    100
mercury monarch ghia                 139
mercury monarch                       72
chevrolet cavalier                    88
Name: horsepower, Length: 196, dtype: int64

In [9]:
intercept = pd.DataFrame({'intercept' : np.ones(X_train.shape[0])})
intercept.index = Auto_train['horsepower'].index
X_train_2 = pd.concat([intercept, Auto_train['horsepower']], axis=1)


In [10]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.577
Method:                 Least Squares   F-statistic:                     267.1
Date:                Tue, 22 Oct 2024   Prob (F-statistic):           2.51e-38
Time:                        21:13:17   Log-Likelihood:                -601.69
No. Observations:                 196   AIC:                             1207.
Df Residuals:                     194   BIC:                             1214.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     40.2760      1.088     37.018      0.000      38.130      42.422
horsepower    -0.1611      0.010    -16.345      0.000      -0.181      -0.142
==============================================================================
Omnibus:                        9.453   Durbin-Watson:                   1.676
Prob(Omnibus):                  0.009   Jarque-Bera (JB):                9.530
Skew:                           0.530   Prob(JB):                      0.00852
Kurtosis:                       3.204   Cond. No.                         321.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now we use the `predict()` method of results evaluated on the model matrix using the validation data set to calcultate the validation MSE of our model.

In [11]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid) # we compute the 196 predictions from the the X_valid df
np.mean((y_valid - valid_pred)**2)

Traceback (most recent call last):
  File "c:\Users\doria\Documents\pythonEnv\dataScience\Lib\site-packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_vars.py", line 622, in change_attr_expression
    value = eval(expression, frame.f_globals, frame.f_locals)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1
    intercept  horsepowername                                           datsun 200sx                    1.0         100amc hornet                      1.0          97ford country squire (sw)        1.0         142amc concord dl 6                1.0          90volkswagen rabbit               1.0          71...                             ...         ...volkswagen dasher               1.0          71datsun pl510                    1.0          88buick skyhawk                   1.0         110fiat x1.9                       1.0          67dodge aries se                  1.0          84[196 rows x 2 columns]
                                     

20.755407959228602

Let's define the validation MSE for higher degree polynomial regressions. We code a function `eval_MSE()` that takes a model string as well as a training set and returns the MSE on the test set.

In [8]:
def evalMSE(terms, response, train, test):
    '''
    Calculate the Mean Squared Error (MSE) between predicted and actual values.

    Parameters:
    ----------
    terms : array-like, shape (n_features,)
        The predictor variables (or features) used in the model.
        
    response : char
        The actual target value (or response) from the dataset.
        
    train : dataFrame
        The training data from the original dataFrame.
        
    test : dataFrame
        The testing data from the original dataFrame

    Returns:
    --------
    mse : float
        The calculated mean squared error on the test data.
    '''

    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]

    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)

    return np.mean((y_test - test_pred)**2)

Let's use this function using linear, quadratic and cubic fits.

In [9]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid
    )
MSE

array([20.75540796, 16.94510676, 16.97437833])

#### Interpretation
While changing random state, the results are consistent with our previous findings:
a model that predicts mpg using a quadratic function of `horsepower` performs better than a model that involves only linear function of `horsepower`. and there is no evidence of an imporvement in using a cubic function of `horsepower`

## Cross Validation